In [ ]:
# classifying statements by first classifying each sentence

import openai
import pandas as pd
import time
from tqdm import tqdm
from dotenv import load_dotenv
import os

import utils
from utils import extract_score

label_to_score = utils.label_to_score

load_dotenv()
openai.api_key=os.getenv("OPENAI_KEY")

In [ ]:
statements = pd.read_csv('../data/statements.csv')

In [ ]:
prompt_template = """
<sentence>
INPUT
</sentence>
<labels>
Dovish: Strongly expresses a belief that the economy may be
growing too slowly and may need stimulus through mon-
etary policy.
Mostly dovish: Overall message expresses a belief that the economy may
be growing too slowly and may need stimulus through
monetary policy.
Neutral: Expresses neither a hawkish nor dovish view and is
mostly objective.
Mostly hawkish: Overall message expresses a belief that the economy is
growing too quickly and may need to be slowed down
through monetary policy.
Hawkish: Strongly expresses a belief that the economy is growing
too quickly and may need to be slowed down through monetary policy.
</labels>
Which label best applies applies to the sentence (Dovish, Mostly Dovish, Neutral, Mostly Hawkish, Hawkish)?
"""

In [ ]:
global_sentence_scores = {}
for i, statement in tqdm(statements.iterrows()):
    text = statement['statements_text']
    sentences = text.split('.')
    sentence_scores = []
    for sentence in sentences:
        if len(sentence) < 5:
            sentence_scores.append("TOO SHORT")
            continue
        # call GPT-4
        response = None
        while response is None:
            try:
                response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=[
                        {"role": "user", "content": prompt_template.replace("INPUT", sentence)},
                    ],
                temperature=0
                )
            except Exception as e:
                print(e)
                time.sleep(1)
        sentence_score = extract_score(response["choices"][0]["message"]["content"])
        sentence_scores.append(sentence_score)
    global_sentence_scores[statement["date"]] = sentence_scores

In [ ]:
global_sentence_scores

In [ ]:
import json
# Specify the file path where you want to save the JSON file
file_path = '../results/statements_scores_by_sentence.json'

# Convert the dictionary to JSON format
json_data = json.dumps(global_sentence_scores)

# Write the JSON data to the file
with open(file_path, 'w') as file:
    file.write(json_data)